In [1]:
import os
import sys
from functools import partial

import numpy as np
import tensorflow as tf
from scipy.stats import beta

sys.path.append(os.path.join("..", "..", ".."))
import cardio.dataset as ds
from cardio import EcgDataset
from cardio.dataset import B, V, F
from cardio.models.dirichlet_model import DirichletModel, concatenate_ecg_batch
from cardio.models.metrics import f1_score, classification_report, confusion_matrix

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/anaconda3/envs/tensorflow/lib/python3.6/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
Using TensorFlow backend.


In [8]:
MODEL_PATH = os.path.expanduser("~")+"/coding/cnn/DeepECG/dirichlet_model"

BATCH_SIZE = 100

In [9]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5, allow_growth=True)

model_config = {
    "session": {"config": tf.ConfigProto(gpu_options=gpu_options)},
    "build": False,
    "load": {"path": MODEL_PATH},
}

In [10]:
template_predict_ppl = (
    ds.Pipeline()
      .init_model("static", DirichletModel, name="dirichlet", config=model_config)
      .init_variable("predictions_list", init_on_each_run=list)
      .load(fmt="wfdb", components=["signal", "meta"])
      .flip_signals()
      .split_signals(2048, 2048)
      .predict_model("dirichlet", make_data=partial(concatenate_ecg_batch, return_targets=False),
                     fetches="predictions", save_to=V("predictions_list"), mode="e")
      .run(batch_size=BATCH_SIZE, shuffle=False, drop_last=False, n_epochs=1, lazy=True)
)

INFO:tensorflow:Restoring parameters from /Users/truongnm/coding/cnn/DeepECG/dirichlet_model/model-26000


Original file

In [13]:
signal_path = os.path.expanduser("~")+"/coding/cnn/ecg/ecg-process/raw/A00001.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [14]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.021257801, 'NO': 0.97874224},
  'uncertainty': 0.007573604583740234}]

MAT -> csv -> MAT file

In [18]:
signal_path = os.path.expanduser("~")+"/coding/cnn/ecg/ecg-process/raw/A00001-convertedm.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [19]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.021257801, 'NO': 0.97874224},
  'uncertainty': 0.007573604583740234}]

DB MAT file -> csv -> signal -> csv 250 lines (1s) -> signal -> MAT file -> test

In [20]:
signal_path = os.path.expanduser("~")+"/coding/cnn/ecg/ecg-process/A00001-converted-1sm.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [21]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.021675922, 'NO': 0.97832406},
  'uncertainty': 0.00739288330078125}]

DB MAT file -> csv -> signal -> csv 250 lines (1s) -> 
gnuplot image - this can be image received from user -> csv 250 lines (1s) -> MAT file -> test

In [22]:
signal_path = os.path.expanduser("~")+"/coding/cnn/ecg/ecg-process/A00001-converted-1s-genfromuserm.hea"
predict_eds = EcgDataset(path=signal_path, no_ext=True, sort=True)
predict_ppl = (predict_eds >> template_predict_ppl).run()

In [23]:
predict_ppl.get_variable("predictions_list")

[{'target_pred': {'A': 0.021675922, 'NO': 0.97832406},
  'uncertainty': 0.00739288330078125}]